# Registers

In memory mapped register ('MMR') decoders there is a need for a certain abstraction in order to maintain adressable registers.

In [1]:
from cyhdl import *
from myirl.library.registers import *

## Bitfield and Register classes

A Register contains a set of bit fields which can have a specific value. For bus decoding, it is often necessary to map a register item to a plain data bus signal.

In [2]:
BF = NamedBitfield
A = BF("im", 3, 1)
B = BF("ex", 7, 6)
C = BF("inv", 4, 4)
D = BF("mode", 14, 10)

r = Register(16, [A, B, C, D])

Dump bit map in MSB->LSB order. Bits listed as `None` are undefined. When reading from a MMR, undefined values may be not asserted to the bus, or assigned to a default value, depeding on the inference rules.

In [3]:
list(reversed(r.layout))

[[None 15:15],
 [mode 14:10],
 [None 9:8],
 [ex 7:6],
 [None 5:5],
 [inv 4:4],
 [im 3:1],
 [None 0:0]]

### Concat composition

To return a concatenation, we use the `.compose` notation below.
We first instance a dummy target module (VHDL by default) to elaborate on the HDL equivalent.

In [4]:
from myirl.targets.dummy import DummyTargetModule

d = DummyTargetModule()

In [5]:
s = Signal(intbv()[3:], name = 's')

c = r.compose(im = s, ex = 3, mode = 5)

q = Signal(intbv()[16:], name = 'q')
a = q.set(c)

a.emit(d)

q <= ('0' & "00101" & "00" & "11" & '0' & '0' & s & '0');


### Partial assignment

The partial assignment is not supported with standard `Signal` types, a `PASignal` must be used instead for the alternative composition, returning generators. Note that a PASignal can **not** be passed through the interface.
Also note that assigning PASignal slices directly (such as `pa[5:3].set(5)` is not fully portable. The proper coding practise is to use PAsignals together in combination with the `register.assign()` method listed below.

In [6]:
p = PASignal(intbv()[16:], name = 'p')

In [7]:
gen = r.assign(p, im = s, ex = 3, mode = 5)

In [8]:
gen.emit(d)

p(3 downto 1) <= s;
p(7 downto 6) <= "11";
p(14 downto 10) <= "00101";


### Issues

Partial assignments cause some issues in internal driver resolving and can under some circumstances create more logic congestion than actually necessary.

The recommended approach is to use the `.compose` method to explicitely assign all bits of a signal. For the register class, the `.undefined` method can be overriden to use other default values:

In [9]:
from myirl.library.dontcare import DontCare

class MyRegister(Register):
    def undefined(self, size):
        return DontCare(size, '?')

In [10]:
r = MyRegister(16, [A, B, C, D])
s = Signal(intbv()[3:], name = 's')

vlog = DummyTargetModule(targets.Verilog)
c = r.compose(im = s, ex = 3, mode = 5)
q.set(c).emit(vlog)

assign  q = {1'b?, 5'b00101, 2'b??, 2'b11, 1'b?, 1'b0, s, 1'b?}; /* fallback */


## Register Signals

Register signals are containers that create a set of input and output signals, according to a passed `Reg` template.
Unlike a `@container(CONTAINER_INTERFACE` class, they use separate members for write and read signals.

It requires an augmented `Register` type from the `soc` library, so a redefinition of the above register is required:

In [11]:
from cyrite.library.soc import RegisterSignal, Reg, BF

r0 = Reg(16,
    [
        BF("im", 3, 1, flags = BF.READONLY),
        BF("ex", 7, 6),
        BF("inv", 4, 4, flags = BF.WRITEONLY),
        BF("mode", 14, 10, default = 2)
    ]
)


rs = RegisterSignal("regsig", template = r0)

When part of an interface, the `read` part of the container serves as inputs, whereas `write` members are outputs. So keep in mind to manipulate the `read` members from the testbench.

In [12]:
rs.read.members()

{'im': <regsig_read.im>, 'ex': <regsig_read.ex>, 'mode': <regsig_read.mode>}

We note that `WRITEONLY` members do not appear in the `read` port.

In [13]:
rs.write.members()

{'ex': <regsig_write.ex>,
 'inv': <regsig_write.inv>,
 'mode': <regsig_write.mode>}

In [14]:
rs.get_children()

{'read': {`<class '__main__.regsig_read'>` | 'im', 'ex', 'mode'},
 'write': {`<class '__main__.regsig_write'>` | 'ex', 'inv', 'mode'}}

### RegisterSignal with sensitivity

Sometimes, logic wants to be notified in particular when a value is written. A register signal therefore can be flagged `WRITEONLY | VOLATILE` as a whole.

Whenever a CPU is writing a new value, a separate `select` pin is pulsed.

Note that `flags` is specified in the register argument. The `VOLATILE` flag is **not** supported for single bit fields, as it was considered deprecated design practise.

In [15]:
r1 = Reg(16,
    [
        BF("data", 7, 0),
    ], flags = Reg.VOLATILE | Reg.WRITEONLY
)

In [16]:
rs = RegisterSignal("regsig", template = r1)
rs.get_children()

{'read': {`<class '__main__.regsig_read'>` | 'data'},
 'write': {`<class '__main__.regsig_write'>` | 'data'},
 'select': {`<class '__main__.regsig_sel'>` | 'sel_w'}}

We can access this select signal as follows:

In [17]:
len(rs.select.sel_w)

1

## Examples

* [Cross clock domain passing of register values](xclkdomain.ipynb#Register-passing)